In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [14]:
#from fastai.imports import *
#from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import numpy as np
import pandas as pd
import re

In [3]:
PATH = 'data/bulldozers/'

In [6]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, 
                 parse_dates=["saledate"])

In [7]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000, 
                           'display.max_columns', 1000):
        display(df)       
    

In [8]:
display_all(df_raw.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [9]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaN,NaN,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaN,NaN,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaN,NaN,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaN,NaN,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaN,NaN,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaN,NaN,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaN,NaN,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaN,NaN,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2011-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# need the log of sale price
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [11]:
?? pd.to_datetime()

In [15]:
# function to define expansion of date time into columns for year, month, week etc.

def add_date_info(df, fldname, drop=True, time=False):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64
    
    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
        
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    
    # attributes to add to dataframe
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 
            'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    # add time attributes if time=True
    if time:
        attr = attr + ['Hour', 'Minute', 'Second']
    # add datetime attributes to the dataframe
    for n in attr:
        df[targ_pre + n] = getattr(fld.dt, n.lower())
    
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10**9
    
    # drop the original date column if drop=True
    if drop:
        df.drop(fldname, axis=1, inplace=True)   
        

In [16]:
add_date_info(df_raw, 'saledate')

In [17]:
display_all(df_raw.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,11.0974,10.9508,9.21034,10.5584,9.30565
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
fiModelDesc,521D,950FII,226,PC120-6E,S175


The categorical variables are all currently strings, which is not very efficient. The following function converts them to pandas categories.

In [13]:
def convert_cats(df):
    for i, j in df.items():
        if is_string_dtype(j):
            df[i] = j.astype('category').cat.as_ordered()

In [14]:
convert_cats(df_raw)

In [15]:
#os.makedirs('tmp1', exist_ok=True)

In [17]:
df_raw.to_feather('tmp1/bulldozers-rawx')

In [19]:
#df_raw = pd.read_feather('tmp1/bulldozers-rawx')

In [ ]:
def fix_missing(df, col, name, na_dict):
    """ Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col) # sets boolean to na's in col and adds to df
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
        return na_dict
    

In [ ]:
def get_sample(df, n):
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

In [ ]:
# fast ai's proc_df function to split off response variable and change df to entirely numeric
def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    if not ignore_flds:
        ignore_flds = []
    if not skip_flds:
        skip_flds = []
    if subset:
        getsample(df, subset)
    else:
        df = df.copy()
    ignored_flds = df.loc[:, ignored_flds]
    df.drop(ignored_flds, axis=1, inplace=True)
    if preproc_fn:
        preproc_fn(df)
    if y_fld is None:
        y = None
    else:
        if not is_numeric_dtype(df[y_fld]):
            df[y_fld] = df[y_fld].cat.codes
            y = df[y_fld].values
            skip_flds += [y_fld]
        df.drop(skip_flds, axis=1, inplace=True)
    
    if na_dict = None:
        na_dict = {}
    else na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items():
        na_dict = fix_missing(df, c, n, max_n_cat)
    
    
        